In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import  ChatPromptTemplate, MessagesPlaceholder

# MessagesPlaceholder에 chat message 내용들을 기록함

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    # memory_key="chat_history", #memory_key의 default값은 "history"임
    return_messages=True # 메모리에 기록되는 내용을 문자열이 아닌 chat message 형식으로 기록하라고 설정
)

prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

def load_memory(_):
    return memory.load_memory_variables({})["history"]

"""
    chain = prompt | llm
    chain.invoke({
        "history=load_memory(),
        "question":"My name is Peter"
    })

    이렇게 할 경우 chain을 호출할 때마다 history 추가해줘야한다는 문제가 있음

    문제를 해결하기 위해 다음과 같이 코드 변경 가능
    chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm
    chain.invoke({
        "question":"My name is Peter"
    })
"""

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question":question})
    memory.save_context({"input":question},{"output":result.content})
    print(result)

In [ ]:
invoke_chain("My name is Peter")

In [ ]:
invoke_chain("What is my name?")